<a href="https://colab.research.google.com/github/dgg32/crop_area/blob/main/Google_Earth_Engine_USA_Crops_Area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run me first

First of all, run the following cell to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=qjXlzqh4zHMdQ8YMEuZaw8XwpGJGn0QMznlZOt_kCvs&tc=4DbAAYadTtNwXMFgdA2oVblr10O1vDCwbr5W9_Nr_iY&cc=943p6W1P3u_J_6NHsTz9GdAhm5DpQ5E2dHmElvhCcKM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJoagDCCu8GMnfIMtHyqRJqoVUYWF6uwN1RyA5mIdizzmT_cZSjPAo4

Successfully saved authorization token.


In [35]:


areaPerPixel = ee.Image.pixelArea()


provinces = ee.FeatureCollection("FAO/GAUL/2015/level2");



def calculate_area (state, target_index, target, startDate, endDate):
    def generate_collection(feature):
      geometry = feature.geometry();
      
      dataset = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(startDate, endDate)).select("cropland").first()
                  
      o_dataset = dataset.clip(geometry);


      image = dataset.eq(target_index).selfMask().multiply(areaPerPixel).divide(1e6).set('system:time_start', startDate);
      
      
      reduce = image.select('cropland').reduceRegion(
                                                reducer= ee.Reducer.sum(),
                                                geometry= geometry,
                                                scale= 30,
                                                bestEffort= True
                                            );
            
      return feature.set(target, reduce.get("cropland"))

    filterLevel1 = ee.Filter.eq('ADM1_NAME', state);
    o_state = provinces.filter(filterLevel1);

    corn_areas = o_state.map(generate_collection)

    results = corn_areas.reduceColumns(ee.Reducer.sum(), [target])


    return results.values(["sum"]).get(0)

tasks = []

name_list = ee.ImageCollection('USDA/NASS/CDL').select("cropland").first().get("cropland_class_names").getInfo()

targets = ["Corn", "Cotton"]

state_list = ee.List(["Washington", "Wisconsin", "Wyoming", "Maine", "Michigan", "Minnesota", "Montana", "New Hampshire",
                      "New York", "North Dakota", "South Dakota", "Vermont", "West Virginia", "Alabama", "Alaska", "Arizona",
                      "Arkansas", "California", "Colorado", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas",
                      "Kentucky", "Louisiana", "Maryland", "Mississippi", "Missouri", "Nebraska", "Nevada", "New Mexico", "Ohio",
                      "Oklahoma", "Oregon", "Pennsylvania", "Tennessee", "Texas", "Utah", "Virginia", "Connecticut", "Delaware",
                      "District of Columbia", "Florida", "Massachusetts", "New Jersey", "North Carolina", "Rhode Island", "South Carolina"])

for target in targets:
    for year in range(2001, 2022):
        startDate = f'{year}-01-01'
        endDate = ee.Date(startDate).advance(1, "year")

        target_index = name_list.index(target) + 1;

        results = state_list.map(lambda state: calculate_area(state, target_index, target, startDate, endDate)).reduce(ee.Reducer.sum())


        task = ee.batch.Export.table.toDrive(collection=ee.FeatureCollection([ee.Feature(None, {"time": startDate, target: results})]),
                                        description=f'{target}_{startDate}_python_crop',
                                        folder='crop_1',
                                        fileFormat='CSV')
        task.start()

        tasks.append(task)


In [44]:
for task in tasks:
    print (task.status())
    #task.cancel()

{'state': 'COMPLETED', 'description': 'Corn_2001-01-01_python_crop', 'creation_timestamp_ms': 1664201320597, 'update_timestamp_ms': 1664201619792, 'start_timestamp_ms': 1664201374807, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/#folders/1On6UQ-mAGIelyG--sQJfdWQO54nQQ6ng'], 'attempt': 1, 'batch_eecu_usage_seconds': 12686.3134765625, 'id': 'VPPZUY645X3JTZ34GDX4RLA5', 'name': 'projects/earthengine-legacy/operations/VPPZUY645X3JTZ34GDX4RLA5'}
{'state': 'COMPLETED', 'description': 'Corn_2002-01-01_python_crop', 'creation_timestamp_ms': 1664201320876, 'update_timestamp_ms': 1664201770154, 'start_timestamp_ms': 1664201398310, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/#folders/1On6UQ-mAGIelyG--sQJfdWQO54nQQ6ng'], 'attempt': 1, 'batch_eecu_usage_seconds': 15225.9736328125, 'id': 'J2YSSMVXUUMTW5PVRKIKCOEQ', 'name': 'projects/earthengine-legacy/operations/J2YSSMVXUUMTW5PVRKIKCOEQ'}
{'state': 'COMPLETED', 'description': 'Corn_